# Estadísticas de series
***

_Autor:_    __Salvador Navas__ <br> _Revisión:_ __17/06/2020__ <br>


__Descripción__:<br>
Estas funciones permiten obtener estadísticas de series temporales principlamente de precipitación

__Cosas a corregir__ <br>



In [ ]:
import numpy as np
import pandas as pd


In [ ]:
def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
def fi_h(Datos, h, umbral=0):
    ##Calcula la probabilidad de que no llueva in un tiempo arbitrario de longitud h
    exist_nan=np.sum(np.isnan(Datos)==True)
    if exist_nan>0:
        Datos[np.isnan(Datos)==True]=0

        result_1 = np.sum(moving_average(Datos.values, h)<=umbral)/(np.sum(Datos.values>=umbral))
        
        Datos[np.isnan(Datos)==True]=1

        result_2 = np.sum(moving_average(Datos.values, h)<=umbral)/(np.sum(Datos.values>=umbral))
        
        result=(result_1+result_2)/2
        
    else:
    
        result=np.sum(moving_average(Datos.values, h)<=umbral)/(np.sum(Datos.values>=umbral))
    return result ##Está bien()

In [ ]:
def fi_DD_v(Datos, h, umbral=0):
    """Calcula la probabilidad de que no llueva in un tiempo arbitrario de 
    longitud h condicionado a  que el siguiente tiempo h tampoco llueva"""
    
    exist_nan=np.sum(np.isnan(Datos)==True)
    
    if exist_nan>0:
        Datos[np.isnan(Datos)==True]=0
        n=0
        for i in range(len(Datos)): 
            if any(Datos.values[i-h:i]>umbral) and any(Datos.values[i:i+h]>umbral):
                continue
            else: 
                n=n+1
        result_1 = 2 - (n/(np.sum(moving_average(Datos.values, h)<=umbral)))
        
        Datos[np.isnan(Datos)==True]=1
        n=0
        for i in range(len(Datos)): 
            if any(Datos.values[i-h:i]>umbral) and any(Datos.values[i:i+h]>umbral):
                continue
            else: 
                n=n+1 
        result_2 = 2 - (n/(np.sum(moving_average(Datos.values, h)<=umbral)))
        
        result=(result_1+result_2)/2
        
    else:
        n=0
        for i in range(len(Datos)): 
            if any(Datos.values[i-h:i]>umbral) and any(Datos.values[i:i+h]>umbral):
                continue
            else: 
                n=n+1
        result = 2 - (n/(np.sum(moving_average(Datos.values, h)<=umbral)))
        
    return result


def fi_WW_v(Datos, h, umbral=0):
    """Calcula la probabilidad de que llueva in un tiempo arbitrario de longitud 
    h condicionado a que el siguiente tiempo h llueva"""
    
    exist_nan=np.sum(np.isnan(Datos)==True)
    
    if exist_nan>0:
        Datos[np.isnan(Datos)==True]=0
        n=0
        for i in range(len(Datos)): 
            if np.nansum(Datos.values[i-h:i]>umbral) and np.nansum(Datos.values[i:i+h]>umbral): n=n+1 

        result_1 = n/(np.sum(moving_average(Datos.values, h)>umbral))
            
        Datos[np.isnan(Datos)==True]=1
        n=0
        for i in range(len(Datos)): 
            if np.nansum(Datos.values[i-h:i]>umbral) and np.nansum(Datos.values[i:i+h]>umbral): n=n+1 

        result_2 = n/(np.sum(moving_average(Datos.values, h)>umbral))
        
        result=(result_1+result_2)/2  
          
    else:
        n=0
        for i in range(len(Datos)): 
            if np.nansum(Datos.values[i-h:i]>umbral) and np.nansum(Datos.values[i:i+h]>umbral): n=n+1 

        result = n/(np.sum(moving_average(Datos.values, h)>umbral))
            
    return result

In [ ]:
def calculate_statistics(Datos,statistics_list,temporal_resolution):
    """Calculo los estaísticos para una estacion determinada
    Los estadísticos que se pueden calcular son: mean, var_h, autocorr_l_h, fih_h, fiWW_h, fiDD_h, M3_h
    
    Parámetros:
    -----------
    Datos:           dataframe. Serie de datos a analizar
    statistics_list:      list. Lista con los estadisticos que se desea analizar. Es necesario añadir el nombre y la agragación temporal a la que se quiere calcular el estadístico.
                                Por ejemplo si la serie está a resolucion diaria y queremos calcular el estadístico de la varianza cada dos días pondremos var_2. 
                                Si es horaria y queremos cada dos horas pondremos lo mismo
    
    temporal_resolution:   int. Resolución temporal a la que se encuentra la serie de partida
    
    Salidas:
    -----------
    Array con los estadisticos que se han introducido.
             

    """

    statististics_values_real=list()  
    
    if temporal_resolution=='d':
        t='D'
    elif temporal_resolution=='h':
        t='h'
    for s in statistics_list:
        statistics=[s]

        if np.sum(['mean' in i for i in statistics])>=1:#Quiere decir que hay que ajustar ese estadístico
            statististics_values_real.append(np.nanmean(Datos))
        if np.sum(['var' in i for i in statistics])>=1:
            pos=np.where(['var' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                aux=Datos.resample(str(h) + t).sum(); 
                statististics_values_real.append(np.nanvar(aux))

        if np.sum(['autocorr' in i for i in statistics])>=1:
            pos=np.where(['autocorr' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                l=int(statistics[ii].split("_",3)[1])
                h=int(statistics[ii].split("_",3)[2])
                aux=Datos.resample(str(h) + t).sum(); 
                Autocorrelation_aux=aux.autocorr(lag=l) 
                if np.size(Autocorrelation_aux)>1: Autocorrelation_aux=Autocorrelation_aux[0] 
                statististics_values_real.append(Autocorrelation_aux)

        if np.sum(['fih' in i for i in statistics])>=1:
            pos=np.where(['fih' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                statististics_values_real.append(fi_h(Datos, h))

        if np.sum(['skewness' in i for i in statistics])>=1:
            pos=np.where(['skewness' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                aux=Datos.resample(str(h) + t).sum(); 
                statististics_values_real.append(sp.stats.moment(aux, moment=3, nan_policy='omit')/(np.nanvar(aux)**(3/2)))

        if np.sum(['fiWW' in i for i in statistics])>=1:
            pos=np.where(['fiWW' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                statististics_values_real.append(fi_WW_v(Datos, h))

        if np.sum(['fiDD' in i for i in statistics])>=1:
            pos=np.where(['fiDD' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                statististics_values_real.append(fi_DD_v(Datos, h))

        if np.sum(['M3' in i for i in statistics])>=1:
            pos=np.where(['M3' in i for i in statistics]); pos=pos[0]
            for i, ii in enumerate(pos):
                h=int(statistics[ii].split("_",1)[1])
                aux=Datos.resample(str(h) + t).sum();
                statististics_values_real.append(sp.stats.moment(aux, moment=3, nan_policy='omit'))

    return statististics_values_real